In [2]:
import os
import json
import re
import csv
from scipy.stats import chi2_contingency

from sklearn.metrics import precision_score, recall_score, f1_score


import numpy as np
import xlrd
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import  metrics
import random


from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix,f1_score,roc_auc_score,roc_curve
import matplotlib.pyplot as plt
import math

/Users/linliu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
from nltk.corpus import stopwords
words = stopwords.words('english')

In [4]:
Train_Fake_Data = []
Train_True_Data = []
Test_Fake_Data = []
Test_True_Data = []

In [5]:
f = open("./data/Fake_train")
for line in f.readlines():
    Train_Fake_Data.append(line)
f.close()

f = open("./data/True_train")
for line in f.readlines():
    Train_True_Data.append(line)
f.close()

f = open("./data/Fake_test")
for line in f.readlines():
    Test_Fake_Data.append(line)
f.close()

f = open("./data/True_test")
for line in f.readlines():
    Test_True_Data.append(line)
f.close()

In [6]:
def feature_importance(dict_names):
    np.random.seed(1)
    print("current dict: ",dict_names)
   
    
    def get_feature(True_data=None, Fake_data=None):
        features_word = ['all', 'propaganda', 'persuasive','sentiment', 'subjectivity', 'technical']
        all_feature_words_list = []
        for i, word in enumerate(features_word):
            true_path = './result_fake_true_after_reduce_stopwords/' + 'true_' + word + '_words.csv'
            fake_path = './result_fake_true_after_reduce_stopwords/' + 'fake_' + word + '_words.csv'
            true_words = open(true_path, encoding='utf-8', mode='r')
            true_words_list = []
            true_words = true_words.readlines()
            for line in true_words:
                line = line.strip('\n\r').split(',')
                # print(line[0])
                true_words_list.append(line[0])
            all_feature_words_list.append(true_words_list)

            fake_words = open(fake_path, encoding='utf-8', mode='r')
            fake_words_list = []
            fake_words = fake_words.readlines()
            for line in fake_words:
                line = line.strip('\n\r').split(',')
                fake_words_list.append(line[0])
                # print(line[0])
            all_feature_words_list.append(fake_words_list)

        #print(len(all_feature_words_list))

        all_data = []
        for line in True_data[:]:
            one_data = [0, ]
            line = line.split(' ')
            words_cnt = 0
            for w in line:
                if w not in words:
                    words_cnt+=1
            for feature in all_feature_words_list:
                num = 0
                for word in line:
                    if word in feature:
                        num += 1
                one_data.append(num)
            one_data.append(words_cnt)
            # print(one_data)
            all_data.append(one_data)

        for line in Fake_data[:]:
            one_data = [1, ]
            line = line.split(' ')
            words_cnt = 0
            for w in line:
                if w not in words:
                    words_cnt+=1
            for feature in all_feature_words_list:
                num = 0
                for word in line:
                    if word in feature:
                        num += 1
                one_data.append(num)
            one_data.append(words_cnt)
            all_data.append(one_data)
            # print(all_data[:3])
        return all_data
    '''
    [0, 2, 0, 16, 5, 0, 0, 2, 0, 1, 0]
    [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
    [1, 1, 0, 3, 0, 1, 0, 2, 0, 0, 0]
    [1, 1, 0, 3, 1, 0, 0, 0, 0, 0, 0]

    '''

       

    train_dataset = get_feature(Train_True_Data, Train_Fake_Data)
    #validation_dataset=get_feature(Train_True_Data[int(len(Train_True_Data)*1):], Train_Fake_Data[int(len(Train_Fake_Data)*0.9):])
    test_dataset = get_feature(Test_True_Data, Test_Fake_Data)
    

    random.shuffle(train_dataset)#why shuffle???
    random.shuffle(test_dataset)
    #random.shuffle(validation_dataset)
    df_train_data = pd.DataFrame(train_dataset)
    for i in range(1,13):
        df_train_data[i+13] = df_train_data[i]/df_train_data[13]
    df_test_data = pd.DataFrame(test_dataset)
    for i in range(1,13):
        df_test_data[i+13] = df_test_data[i]/df_test_data[13]
    for i in range(6):
        df_train_data[26+i] = df_train_data[2*i+1]-df_train_data[2*i+2]
        df_test_data[26+i] = df_test_data[2*i+1]-df_test_data[2*i+2]
#     df_train_data[26+i] = df_train_data[2*i+1]<df_train_data[2*i+2]
#     df_test_data[26+i] = df_test_data[2*i+1]<df_test_data[2*i+2]

#     print("num of train %d"%(len(train_dataset)))
#     print("num of validation %d"%(len(validation_dataset)))
#     print("num of test %d"%(len(test_dataset)))
    data = np.array(df_train_data) 
    # print(data)
    x = data[:, 1:]  
    y = data[:, 0].astype(int)  
    test_data = np.array(df_test_data)
    test_x = test_data[:, 1:]  
    test_y = test_data[:, 0].astype(int)  
    

    num_train = x.shape[0]
    num_each_part = num_train//5
    #print("num of train data is %s and each of part(1/5) is %s "%(num_train,num_each_part))
    test_accuracy_list = list()
    test_f1_list = list()
    test_precision_list=list()
    test_recall_list=list()
    for i in range(5):
        #print(num_each_part*i,num_each_part*(i+1))
        train_x = x[num_each_part*i:num_each_part*(i+1)]
        train_y = y[num_each_part*i:num_each_part*(i+1)]
        gbdt = GradientBoostingClassifier(max_depth=6,
                                          random_state=0,
                                          min_samples_split=10,
                                          learning_rate=0.01,
                                          n_estimators=200,
                                          subsample=0.85)
        gbdt.fit(train_x,train_y)
        train_score_rbf = gbdt.score(train_x, train_y)
        #print("The train accuracy score of GBDT is : %f" % train_score_rbf)
        test_score_rbf = gbdt.score(test_x, test_y)
        #print("The test accuracy score of GBDT is : %f" % test_score_rbf)
        predict_y = gbdt.predict(test_x)
        prob_y = gbdt.predict_proba(test_x)[:,1]
        from sklearn.metrics import confusion_matrix,f1_score
        f1 = f1_score(test_y, predict_y, average='binary')
        #print("f1:", f1 )
        p = precision_score(test_y, predict_y, average='binary')
        #print("precision:", p)
        r = recall_score(test_y, predict_y,average='binary')
        #print("recall:", r)

        test_accuracy_list.append(test_score_rbf)
        test_f1_list.append(f1)
        test_precision_list.append(p)
        test_recall_list.append(r)
    #print("avg of accuracy of five-fold cross validation is %f"%(np.mean(np.asarray(test_accuracy_list))))
    print("avg of precision of five-fold cross validation is %f"%(np.mean(np.asarray(test_precision_list))))
    print("avg of recall of five-fold cross validation is %f"%(np.mean(np.asarray(test_recall_list))))
    print("avg of f1 of five-fold cross validation is %f"%(np.mean(np.asarray(test_f1_list))))



In [15]:
all_dict = ['persuasive','sentiment','propaganda','subjectivity','technical','all']
dict_num = len(all_dict)
sub_list_all = [] 
for i in range(1 << dict_num):  
    combo_list = [] 
    for j in range(dict_num):
        if i & (1 << j):  
            combo_list.append(all_dict[j])  
    sub_list_all.append(combo_list)
#print(sub_list_all)
for ele in sub_list_all[1:]:
    dict_names = list()
    for i in ele:
        dict_names.extend(["fake_"+i, "true_"+i])
    feature_importance(dict_names)
feature_importance(['fake_subjectivity', 'true_subjectivity'])


print(sub_list_all)

current dict:  ['fake_persuasive', 'true_persuasive']
avg of precision of five-fold cross validation is 0.704597
avg of recall of five-fold cross validation is 0.589720
avg of f1 of five-fold cross validation is 0.641701
current dict:  ['fake_sentiment', 'true_sentiment']
avg of precision of five-fold cross validation is 0.699211
avg of recall of five-fold cross validation is 0.593458
avg of f1 of five-fold cross validation is 0.641290
current dict:  ['fake_persuasive', 'true_persuasive', 'fake_sentiment', 'true_sentiment']
avg of precision of five-fold cross validation is 0.707573
avg of recall of five-fold cross validation is 0.592523
avg of f1 of five-fold cross validation is 0.644795
current dict:  ['fake_propaganda', 'true_propaganda']
avg of precision of five-fold cross validation is 0.696979
avg of recall of five-fold cross validation is 0.593458
avg of f1 of five-fold cross validation is 0.640781
current dict:  ['fake_persuasive', 'true_persuasive', 'fake_propaganda', 'true_pro

avg of precision of five-fold cross validation is 0.708541
avg of recall of five-fold cross validation is 0.589720
avg of f1 of five-fold cross validation is 0.642019
current dict:  ['fake_all', 'true_all']
avg of precision of five-fold cross validation is 0.709207
avg of recall of five-fold cross validation is 0.575701
avg of f1 of five-fold cross validation is 0.634696
current dict:  ['fake_persuasive', 'true_persuasive', 'fake_all', 'true_all']
avg of precision of five-fold cross validation is 0.706138
avg of recall of five-fold cross validation is 0.585981
avg of f1 of five-fold cross validation is 0.640289
current dict:  ['fake_sentiment', 'true_sentiment', 'fake_all', 'true_all']
avg of precision of five-fold cross validation is 0.722083
avg of recall of five-fold cross validation is 0.592523
avg of f1 of five-fold cross validation is 0.650632
current dict:  ['fake_persuasive', 'true_persuasive', 'fake_sentiment', 'true_sentiment', 'fake_all', 'true_all']
avg of precision of five

avg of precision of five-fold cross validation is 0.722245
avg of recall of five-fold cross validation is 0.600000
avg of f1 of five-fold cross validation is 0.654547
current dict:  ['fake_propaganda', 'true_propaganda', 'fake_subjectivity', 'true_subjectivity', 'fake_technical', 'true_technical', 'fake_all', 'true_all']
avg of precision of five-fold cross validation is 0.715978
avg of recall of five-fold cross validation is 0.579439
avg of f1 of five-fold cross validation is 0.640213
current dict:  ['fake_persuasive', 'true_persuasive', 'fake_propaganda', 'true_propaganda', 'fake_subjectivity', 'true_subjectivity', 'fake_technical', 'true_technical', 'fake_all', 'true_all']
avg of precision of five-fold cross validation is 0.713747
avg of recall of five-fold cross validation is 0.599065
avg of f1 of five-fold cross validation is 0.650938
current dict:  ['fake_sentiment', 'true_sentiment', 'fake_propaganda', 'true_propaganda', 'fake_subjectivity', 'true_subjectivity', 'fake_technical',